In [5]:
from scipy.stats import rankdata
import scipy as sp
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv('./alldata.csv')

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
price = df
price

,Stkcd,Trddt,Opnprc,Hiprc,Loprc,Clsprc,Dnshrtrd,Dnvaltrd,Dsmvosd,Dsmvtll,Dretwd,Dretnd,Adjprcwd,Adjprcnd,Markettype,Capchgdt,Trdsta,IndustryCode,IndustryName
0,1,2010-01-04,24.520,24.580,23.680,23.710,24192276,5.802495e+08,69330749.18,73629834.50,-0.027082,-0.027082,818.361203,710.377219,4,2009-10-15,1,480101.0,����
1,1,2010-01-05,23.750,23.900,22.750,23.300,55649982,1.293477e+09,68131862.33,72356606.65,-0.017292,-0.017292,804.209870,698.093176,4,2009-10-15,1,480101.0,����
2,1,2010-01-06,23.250,23.250,22.720,22.900,41214313,9.444537e+08,66962216.62,71114433.15,-0.017167,-0.017167,790.403692,686.108744,4,2009-10-15,1,480101.0,����
3,1,2010-01-07,22.900,23.050,22.400,22.650,35533685,8.041663e+08,66231188.06,70338074.71,-0.010917,-0.010917,781.774831,678.618474,4,2009-10-15,1,480101.0,����
4,1,2010-01-08,22.500,22.750,22.350,22.600,28854306,6.506674e+08,66084982.34,70182803.02,-0.002208,-0.002208,780.049059,677.120420,4,2009-10-15,1,480101.0,����
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8672604,900957,2022-01-24,0.633,0.638,0.615,0.621,479100,2.998840e+05,114264.00,216729.00,-0.026646,-0.026646,0.625923,0.620999,2,2000-08-03,1,620201.0,װ��װ��
8672605,900957,2022-01-25,0.617,0.620,0.607,0.611,372779,2.281030e+05,112424.00,213239.00,-0.016103,-0.016103,0.615844,0.610999,2,2000-08-03,1,620201.0,װ��װ��
8672606,900957,2022-01-26,0.614,0.617,0.610,0.614,222600,1.362910e+05,112976.00,214286.00,0.004910,0.004910,0.618868,0.613999,2,2000-08-03,1,620201.0,װ��װ��
8672607,900957,2022-01-27,0.610,0.613,0.605,0.612,245100,1.487580e+05,112608.00,213588.00,-0.003257,-0.003257,0.616852,0.611999,2,2000-08-03,1,620201.0,װ��װ��


In [10]:
df_open = df.pivot(index=['Trddt'], columns='Stkcd', values=['Opnprc'])
df_open

Opnprc                                                          \
Stkcd      1      2      4      5      6      7      8      9      10       
Trddt                                                                       
2010-01-04  24.52  10.85    NaN   6.01  11.33   7.09    NaN  11.00  11.95   
2010-01-05  23.75  10.51    NaN   5.95  11.12   6.94    NaN  10.89  11.81   
2010-01-06  23.25  10.35    NaN   6.13  10.79   7.17    NaN  10.56  11.61   
2010-01-07  22.90  10.36    NaN   6.03  10.85   7.16    NaN  10.44  11.73   
2010-01-08  22.50  10.28    NaN   5.80  10.70   6.94    NaN  10.70  11.15   
...           ...    ...    ...    ...    ...    ...    ...    ...    ...   
2022-01-24  17.34  21.59  21.00   2.10   4.50   7.18   2.63  14.00   3.70   
2022-01-25  17.08  21.60  21.10   2.11   4.47   6.93   2.65  14.28   3.59   
2022-01-26  16.95  20.99  22.26   2.05   4.31   6.88   2.56  13.61   3.40   
2022-01-27  16.50  20.53  23.00   2.13   4.33   6.51   2.56  13.29   3.43   
2022-01-28  16.39  20.60  20.00   2.07   4.26   6.29   2.52  12.99   3.35   

                   ...                                                   \
Stkcd      11      ... 900947 900948 900949 900950 900951 900952 900953   
Trddt              ...                                                    
2010-01-04  10.79  ...  0.843  8.660  0.708  0.837  0.541  0.484  0.727   
2010-01-05  10.60  ...  0.837  8.758  0.697  0.809  0.548  0.481  0.720   
2010-01-06  10.27  ...  0.840  9.019  0.704  0.818  0.540  0.485  0.739   
2010-01-07  10.18  ...  0.835  8.936  0.700  0.830  0.537  0.481  0.751   
2010-01-08  10.01  ...  0.822  8.770  0.693  0.826  0.528  0.475  0.745   
...           ...  ...    ...    ...    ...    ...    ...    ...    ...   
2022-01-24  11.49  ...  0.282  0.905    NaN    NaN    NaN  0.246  0.334   
2022-01-25  11.42  ...  0.278  0.908    NaN    NaN    NaN  0.244  0.333   
2022-01-26  10.96  ...  0.272  0.882    NaN    NaN    NaN  0.244  0.328   
2022-01-27  11.11  ...  0.271  0.892    NaN    NaN    NaN  0.247  0.322   
2022-01-28  10.86  ...  0.271  0.915    NaN    NaN    NaN  0.245  0.326   

                                 
Stkcd      900955 900956 900957  
Trddt                            
2010-01-04  0.716  0.812  0.559  
2010-01-05  0.703  0.801  0.548  
2010-01-06  0.695  0.813  0.548  
2010-01-07  0.709  0.806  0.542  
2010-01-08  0.705  0.790  0.527  
...           ...    ...    ...  
2022-01-24  0.120    NaN  0.633  
2022-01-25  0.117    NaN  0.617  
2022-01-26  0.116    NaN  0.614  
2022-01-27  0.113    NaN  0.610  
2022-01-28  0.111    NaN  0.615  

[2936 rows x 4824 columns]

In [11]:
df_open = df_open['Opnprc']
df_open

Stkcd,1,2,4,5,6,7,8,9,10,11,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
Trddt,,,,,,,,,,,,,,,,,,,,,
2010-01-04,24.52,10.85,NaN,6.01,11.33,7.09,NaN,11.00,11.95,10.79,...,0.843,8.660,0.708,0.837,0.541,0.484,0.727,0.716,0.812,0.559
2010-01-05,23.75,10.51,NaN,5.95,11.12,6.94,NaN,10.89,11.81,10.60,...,0.837,8.758,0.697,0.809,0.548,0.481,0.720,0.703,0.801,0.548
2010-01-06,23.25,10.35,NaN,6.13,10.79,7.17,NaN,10.56,11.61,10.27,...,0.840,9.019,0.704,0.818,0.540,0.485,0.739,0.695,0.813,0.548
2010-01-07,22.90,10.36,NaN,6.03,10.85,7.16,NaN,10.44,11.73,10.18,...,0.835,8.936,0.700,0.830,0.537,0.481,0.751,0.709,0.806,0.542
2010-01-08,22.50,10.28,NaN,5.80,10.70,6.94,NaN,10.70,11.15,10.01,...,0.822,8.770,0.693,0.826,0.528,0.475,0.745,0.705,0.790,0.527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,17.34,21.59,21.00,2.10,4.50,7.18,2.63,14.00,3.70,11.49,...,0.282,0.905,NaN,NaN,NaN,0.246,0.334,0.120,NaN,0.633
2022-01-25,17.08,21.60,21.10,2.11,4.47,6.93,2.65,14.28,3.59,11.42,...,0.278,0.908,NaN,NaN,NaN,0.244,0.333,0.117,NaN,0.617
2022-01-26,16.95,20.99,22.26,2.05,4.31,6.88,2.56,13.61,3.40,10.96,...,0.272,0.882,NaN,NaN,NaN,0.244,0.328,0.116,NaN,0.614


In [12]:
df_open = df_open.reset_index()

In [13]:
df_open.columns

Index(['Trddt',       1,       2,       4,       5,       6,       7,       8,
             9,      10,
       ...
        900947,  900948,  900949,  900950,  900951,  900952,  900953,  900955,
        900956,  900957],
      dtype='object', name='Stkcd', length=4825)

In [14]:
df_open

Stkcd,Trddt,1,2,4,5,6,7,8,9,10,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
0,2010-01-04,24.52,10.85,NaN,6.01,11.33,7.09,NaN,11.00,11.95,...,0.843,8.660,0.708,0.837,0.541,0.484,0.727,0.716,0.812,0.559
1,2010-01-05,23.75,10.51,NaN,5.95,11.12,6.94,NaN,10.89,11.81,...,0.837,8.758,0.697,0.809,0.548,0.481,0.720,0.703,0.801,0.548
2,2010-01-06,23.25,10.35,NaN,6.13,10.79,7.17,NaN,10.56,11.61,...,0.840,9.019,0.704,0.818,0.540,0.485,0.739,0.695,0.813,0.548
3,2010-01-07,22.90,10.36,NaN,6.03,10.85,7.16,NaN,10.44,11.73,...,0.835,8.936,0.700,0.830,0.537,0.481,0.751,0.709,0.806,0.542
4,2010-01-08,22.50,10.28,NaN,5.80,10.70,6.94,NaN,10.70,11.15,...,0.822,8.770,0.693,0.826,0.528,0.475,0.745,0.705,0.790,0.527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2931,2022-01-24,17.34,21.59,21.00,2.10,4.50,7.18,2.63,14.00,3.70,...,0.282,0.905,NaN,NaN,NaN,0.246,0.334,0.120,NaN,0.633
2932,2022-01-25,17.08,21.60,21.10,2.11,4.47,6.93,2.65,14.28,3.59,...,0.278,0.908,NaN,NaN,NaN,0.244,0.333,0.117,NaN,0.617
2933,2022-01-26,16.95,20.99,22.26,2.05,4.31,6.88,2.56,13.61,3.40,...,0.272,0.882,NaN,NaN,NaN,0.244,0.328,0.116,NaN,0.614
2934,2022-01-27,16.50,20.53,23.00,2.13,4.33,6.51,2.56,13.29,3.43,...,0.271,0.892,NaN,NaN,NaN,0.247,0.322,0.113,NaN,0.610


In [15]:
df_open.mean(axis=1)

0       15.861567
1       15.874498
2       15.996408
3       15.989290
4       15.596966
          ...    
2931    25.750472
2932    25.899940
2933    25.223211
2934    25.275190
2935    24.647835
Length: 2936, dtype: float64

In [16]:
df_open.mean()

Stkcd
1         14.228678
2         17.624193
4         19.239490
5          4.093732
6          6.571747
            ...    
900952     0.426805
900953     0.642098
900955     0.384115
900956     1.199219
900957     0.696365
Length: 4824, dtype: float64

In [17]:
df_open.fillna(df_open.mean(), inplace=True)

In [18]:
df_open

Stkcd,Trddt,1,2,4,5,6,7,8,9,10,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
0,2010-01-04,24.52,10.85,19.23949,6.01,11.33,7.09,8.723644,11.00,11.95,...,0.843,8.660,0.708000,0.837000,0.541000,0.484,0.727,0.716,0.812000,0.559
1,2010-01-05,23.75,10.51,19.23949,5.95,11.12,6.94,8.723644,10.89,11.81,...,0.837,8.758,0.697000,0.809000,0.548000,0.481,0.720,0.703,0.801000,0.548
2,2010-01-06,23.25,10.35,19.23949,6.13,10.79,7.17,8.723644,10.56,11.61,...,0.840,9.019,0.704000,0.818000,0.540000,0.485,0.739,0.695,0.813000,0.548
3,2010-01-07,22.90,10.36,19.23949,6.03,10.85,7.16,8.723644,10.44,11.73,...,0.835,8.936,0.700000,0.830000,0.537000,0.481,0.751,0.709,0.806000,0.542
4,2010-01-08,22.50,10.28,19.23949,5.80,10.70,6.94,8.723644,10.70,11.15,...,0.822,8.770,0.693000,0.826000,0.528000,0.475,0.745,0.705,0.790000,0.527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2931,2022-01-24,17.34,21.59,21.00000,2.10,4.50,7.18,2.630000,14.00,3.70,...,0.282,0.905,0.592559,0.738369,0.607353,0.246,0.334,0.120,1.199219,0.633
2932,2022-01-25,17.08,21.60,21.10000,2.11,4.47,6.93,2.650000,14.28,3.59,...,0.278,0.908,0.592559,0.738369,0.607353,0.244,0.333,0.117,1.199219,0.617
2933,2022-01-26,16.95,20.99,22.26000,2.05,4.31,6.88,2.560000,13.61,3.40,...,0.272,0.882,0.592559,0.738369,0.607353,0.244,0.328,0.116,1.199219,0.614
2934,2022-01-27,16.50,20.53,23.00000,2.13,4.33,6.51,2.560000,13.29,3.43,...,0.271,0.892,0.592559,0.738369,0.607353,0.247,0.322,0.113,1.199219,0.610


In [29]:
std = df_open.std(axis=1)
mean = df_open.mean(axis=1)
mean

0       27.888514
1       27.884405
2       27.933353
3       27.921725
4       27.781318
          ...    
2931    25.475193
2932    25.606084
2933    24.937605
2934    24.987060
2935    24.367124
Length: 2936, dtype: float64

In [85]:
std

0       39.198982
1       39.210659
2       39.208412
3       39.210787
4       39.210491
          ...    
2931    52.028802
2932    52.392327
2933    52.051452
2934    52.434268
2935    51.397427
Length: 2936, dtype: float64

In [79]:
def change_extreme(row):
    m=row[1:].mean()
    sd=row[1:].std()
    row[1:]=row[1:].apply(lambda x:x if m-3*sd<=x<=m+3*sd else (m+3*sd if x>m+3*sd else m-3*sd))
    return row

In [80]:
df_open.apply(change_extreme, axis=1)

Stkcd,Trddt,1,2,4,5,6,7,8,9,10,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
0,2010-01-04,24.52,10.85,19.23949,6.01,11.33,7.09,8.723644,11.00,11.95,...,0.843,8.660,0.708000,0.837000,0.541000,0.484,0.727,0.716,0.812000,0.559
1,2010-01-05,23.75,10.51,19.23949,5.95,11.12,6.94,8.723644,10.89,11.81,...,0.837,8.758,0.697000,0.809000,0.548000,0.481,0.720,0.703,0.801000,0.548
2,2010-01-06,23.25,10.35,19.23949,6.13,10.79,7.17,8.723644,10.56,11.61,...,0.840,9.019,0.704000,0.818000,0.540000,0.485,0.739,0.695,0.813000,0.548
3,2010-01-07,22.90,10.36,19.23949,6.03,10.85,7.16,8.723644,10.44,11.73,...,0.835,8.936,0.700000,0.830000,0.537000,0.481,0.751,0.709,0.806000,0.542
4,2010-01-08,22.50,10.28,19.23949,5.80,10.70,6.94,8.723644,10.70,11.15,...,0.822,8.770,0.693000,0.826000,0.528000,0.475,0.745,0.705,0.790000,0.527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2931,2022-01-24,17.34,21.59,21.00000,2.10,4.50,7.18,2.630000,14.00,3.70,...,0.282,0.905,0.592559,0.738369,0.607353,0.246,0.334,0.120,1.199219,0.633
2932,2022-01-25,17.08,21.60,21.10000,2.11,4.47,6.93,2.650000,14.28,3.59,...,0.278,0.908,0.592559,0.738369,0.607353,0.244,0.333,0.117,1.199219,0.617
2933,2022-01-26,16.95,20.99,22.26000,2.05,4.31,6.88,2.560000,13.61,3.40,...,0.272,0.882,0.592559,0.738369,0.607353,0.244,0.328,0.116,1.199219,0.614
2934,2022-01-27,16.50,20.53,23.00000,2.13,4.33,6.51,2.560000,13.29,3.43,...,0.271,0.892,0.592559,0.738369,0.607353,0.247,0.322,0.113,1.199219,0.610


In [84]:
df_open.iloc[0, 1:].sort_values()

Stkcd
900938         0.392
900919         0.394
900937         0.395
900939         0.405
900946         0.427
             ...    
688029    145.485459
603501    145.485459
688027    145.485459
300896    145.485459
301031    145.485459
Name: 0, Length: 4824, dtype: object